In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
%matplotlib inline
import glob
from astropy.io.fits import getdata, getheader

In [2]:
# Setup NuSTAR time epochs:
launch_met=77241600. # 2012-06-13T00:00:00

from astropy.convolution import convolve, Box1DKernel


In [3]:
# Step one, sort all of the data:

for ind, file in enumerate(glob.glob('../full_mission/*A_02*')):
    # Load the header:
    hdr = getheader(file, 1)
#    dt_weeks = np.floor((hdr['TSTART'] - launch_met) / (7 * 86400.))
#    print(dt_weeks)
    stub= {'TSTART':np.float(hdr['TSTART']),
           'FNAME':file,
           'DATE-OBS':hdr['DATE-OBS']}
    if(ind==0):
        dbase = [stub]
    else:
        dbase.append(stub)


    



In [4]:
def show_map2(map, exposure):
    fig = plt.figure(figsize=(14, 6))
    ax = fig.add_subplot(111, title='imshow: square bins')
    plt.imshow(
                map,
                interpolation='nearest', origin='low',
                cmap=cm.jet, extent=[0, 4096, -500, 500],
                aspect=8,
                norm=colors.LogNorm(vmin=10.0, vmax = 100.))
    plt.colorbar()


    plt.show()


In [5]:
df = pd.DataFrame(dbase)
df_sorted = df.sort_values(by='TSTART')

interval = 43.*86400. # Weekly
reset_interval=1

nlatbins = 1000
nlonbins = 4096 

In [6]:
for ind, file in enumerate(df_sorted['FNAME']):
    
    if (ind < 100):
        continue
    evt = getdata(file, 1)
    hdr = getheader(file, 1)
    if(reset_interval == 1):
        tmin = evt['TIME'].min()

    time_filter=( (evt['TIME'] - tmin) < interval).nonzero()

    thismap, xedges, yedges = np.histogram2d(evt['PI'][time_filter],
                             evt['SURRPI'][time_filter],
                             range = [[0, 4096], [-500, 500]],
                             bins=[nlonbins, nlatbins])
#    plt.scatter(evt['LON'][time_filter], evt['LAT'][time_filter])

    if(reset_interval == 1):
        tmin = evt['TIME'].min()
        reset_interval=0
        allmap = thismap
        exposure = np.zeros_like(allmap)
    else:
        allmap+=thismap  
        filled = (thismap > 0).nonzero()
        exposure[filled] += 1.0

    if(max(evt['TIME']) > tmin + interval):    
        filled = (exposure > 0).nonzero()
        allmap[filled] = allmap[filled] / exposure[filled]

        plotmap = allmap.T
        plotexp = exposure.T

        empty =(plotmap == 0).nonzero()
        plotmap[empty]=np.nan
        plotexp[empty] = np.nan

        show_map2(plotmap, plotexp)
        reset_interval=1
    
    if ind > 700:
         break
#        break


KeyError: "Key 'SURRPI' does not exist."

In [ ]:
def show_map(map, exposure):
    fig = plt.figure(figsize=(14, 6))
    ax = fig.add_subplot(211, title='imshow: square bins')
    plt.imshow(
                map,
                interpolation='nearest', origin='low',
                cmap=cm.jet, extent=[0, 360, -6.5, 6.5],
                aspect=8,
                norm=colors.LogNorm(vmin=1.0, vmax = 100.))
    plt.colorbar()

    ax = fig.add_subplot(212, title='imshow: square bins')
#     plt.show()

    plt.imshow(
                exposure,
                interpolation='nearest', origin='low',
                cmap=cm.jet, extent=[0, 360, -6.5, 6.5],
                aspect=8,vmin=1.0, vmax =5.)
    plt.colorbar()
    plt.show()
#     plt.close(fig)
    input("Press Enter to continue...")
    fig.clf()    

#plt.set_ytick(yedges)

#plt.gca().set_yticks(yedges)
# ax = fig.add_subplot(132, title='pcolormesh: actual edges',
#         aspect='equal')
# X, Y = np.meshgrid(xedges, yedges)
# ax.pcolormesh(X, Y, thismap)

#ax = fig.add_subplot(133, title='Scatter', aspect = 15)
#plt.scatter(evt['LON'][time_filter], evt['LAT'][time_filter])
#plt.xlim(0, 360)
#plt.ylim(-12, 12)


In [7]:
evt.columns

ColDefs(
    name = 'TIME'; format = 'D'
    name = 'DET_ID'; format = 'I'
    name = 'PI'; format = 'I'
    name = 'RAWX'; format = 'I'
    name = 'RAWY'; format = 'I'
    name = 'DEPTHFLAG'; format = 'I'
    name = 'GRADE'; format = 'I'
    name = 'GEOMAG'; format = 'E'
    name = 'SUNSHINE'; format = 'I'
    name = 'RAM_ANGLE'; format = 'E'
    name = 'LAT'; format = 'E'
    name = 'LON'; format = 'E'
    name = 'ALT'; format = 'E'
    name = 'LIMB_ANGLE'; format = 'E'
)